In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [38]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [16]:
import pandas as pd
import numpy as np
import re
import string
import time

In [27]:
import os
base_dir = '/content/drive/My Drive/NLP/'
os.chdir(base_dir)

In [28]:
data = pd.read_csv('data/chinease_news_train_data.csv')
data.head(3)

,Unnamed: 0,text,category
0,1497,﻿ 日月 光华 - - Traffic _ Info 精华区 文章 阅读- - - ...,traffic
1,436,﻿ 日本 去年 海外 资产 和 负债额 均 创 ...,economic
2,736,﻿ 梁斌 黄胄 联袂 在 京 举办 画展新华社 ...,art


In [29]:
data = data.iloc[:,1:]
data.head(3)

,text,category
0,﻿ 日月 光华 - - Traffic _ Info 精华区 文章 阅读- - - ...,traffic
1,﻿ 日本 去年 海外 资产 和 负债额 均 创 ...,economic
2,﻿ 梁斌 黄胄 联袂 在 京 举办 画展新华社 ...,art


In [33]:
with open('chinese_stopwords.txt', 'r') as f:
    #list of all stopwords
    stopwords = f.readlines()

with open('chinese_numbers_punctuation.txt', 'r') as f:
    #string with all chinese number and punctuations
    num_punct = f.read()
    
    
def get_cleaned_text(input_str):
    
    # to remove punctuations and numbers
    cleaned_words_list = input_str.translate(str.maketrans('','',string.punctuation + string.digits + num_punct + '\ufeff')).split()

    # to remove stop words
    tokens = []
    for word in cleaned_words_list:
        if word not in stopwords:
            tokens.append(word)
    return ' '.join(tokens)

In [34]:
data['cleaned_text'] = data['text'].apply(get_cleaned_text)

In [35]:
data.drop(columns = 'text', inplace=True)
data.head(3)

,category,cleaned_text
0,traffic,月 光华 Traffic Info 精华区 文章 阅读 发信人 RaoRn 亮马 亮马 信 ...
1,economic,本 去年 海外 资产 和 负债额 均 创 历史纪录新华社 东京 月 电 本 内阁 今天 发表...
2,art,梁斌 黄胄 联袂 在 京 举办 画展新华社 北京 月 电 记者 尹鸿祝 河北省 蠡县梁庄 两...


In [36]:
labels_list = sorted(list(set(data.category.values)))
labels_dict = dict([(category,index) for index, category in enumerate(labels_list)])
print(labels_dict)

data['category'] = data['category'].replace(labels_dict)

{'art': 0, 'computer': 1, 'economic': 2, 'education': 3, 'environment': 4, 'medical': 5, 'military': 6, 'politics': 7, 'sports': 8, 'traffic': 9}


In [37]:
data.head(3)

,category,cleaned_text
0,9,月 光华 Traffic Info 精华区 文章 阅读 发信人 RaoRn 亮马 亮马 信 ...
1,2,本 去年 海外 资产 和 负债额 均 创 历史纪录新华社 东京 月 电 本 内阁 今天 发表...
2,0,梁斌 黄胄 联袂 在 京 举办 画展新华社 北京 月 电 记者 尹鸿祝 河北省 蠡县梁庄 两...


In [44]:
sentences = data.cleaned_text.values
labels = data.category.values

In [42]:
!pip install -q transformers

In [69]:
import torch
from transformers import BertTokenizer

bert_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
input_ids = []
# attention_masks = []
sent_length =[]

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(           # encode_plus is better than encode function it can return attention_mask as well
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        # max_length = 510,           # Pad & truncate all sentences.
                        # pad_to_max_length = True,
                        return_attention_mask = False, # Construct attn. masks.
                        # return_tensors = 'pt',    # Return pytorch tensors.
                        verbose = True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    # attention_masks.append(encoded_dict['attention_mask'])

    # store length of each sentences to get the overview of the size of document
    sent_length.append(len(encoded_dict['input_ids']))

In [60]:
max_sent_length = max(sent_length)
min_sent_length = min(sent_length)
median_sent_length =  int(np.median(sent_length))
print('sentence length stats ->', '\nmax:', max_sent_length, '\nmin:',min_sent_length, '\nmedian',median_sent_length)

sentence length stats -> 
max: 23946 
min: 47 
median 396


In [61]:
count_gtr_than_median = sum([1 if i>median_sent_length else 0 for i in sent_length])

print('% of document with length > median words count is:', round(count_gtr_than_median/len(sent_length)*100,2))

% of document with length > median words count is: 49.92


In [62]:
from keras.preprocessing.sequence import pad_sequences

input_ids = pad_sequences(input_ids, maxlen = median_sent_length, value = 0, truncating = 'post', padding = 'post')

In [63]:
# we need attention mask to tell model that which tokens are actual words and which are padding
# here we will put 1 if that token is word and 0 if it is [PAD]

attention_masks = []

for sent in input_ids:

    mask = [int(token_id > 0) for token_id in sent]

    attention_masks.append(mask)


In [64]:
from sklearn.model_selection import train_test_split

train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(input_ids, labels, random_state = 12, test_size = 0.1)

train_masks, valid_masks,_,_ = train_test_split(attention_masks, labels, random_state = 12, test_size = 0.1)

In [65]:
train_inputs =  torch.tensor(train_inputs)
valid_inputs =  torch.tensor(valid_inputs)
train_labels =  torch.tensor(train_labels)
valid_labels =  torch.tensor(valid_labels)
train_masks =   torch.tensor(train_masks)
valid_masks =   torch.tensor(valid_masks)

In [66]:
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler


batch_size = 4

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)


valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = RandomSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler = valid_sampler, batch_size = batch_size)

In [70]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
        bert_model_name,
        num_labels = len(labels_list),
        output_attentions = False,
        output_hidden_states = False

)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [82]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (21128, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [83]:
lr = 5e-5
epochs = 4

optimizer = AdamW(model.parameters(),
                  lr = lr,
                  eps = 1e-8 
                )


In [84]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [85]:
# Helper function to summarise training process

import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [91]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 100 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        b_input_ids = torch.tensor(b_input_ids).to(device).long()
        b_input_mask = torch.tensor(b_input_mask).to(device).long()
        b_labels = torch.tensor(b_labels).to(device).long()
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        loss = outputs[0]

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]


        b_input_ids = torch.tensor(b_input_ids).to(device).long()
        b_input_mask = torch.tensor(b_input_mask).to(device).long()
        b_labels = torch.tensor(b_labels).to(device).long()
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the valid loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this valid run.
    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(valid_dataloader)
    
    # Measure how long the valid run took.
    valid_time = format_time(time.time() - t0)
    
    print("  valid Loss: {0:.2f}".format(avg_val_loss))
    print("  valid took: {:}".format(valid_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'valid Time': valid_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   100  of    563.    Elapsed: 0:00:20.
  Batch   200  of    563.    Elapsed: 0:00:40.
  Batch   300  of    563.    Elapsed: 0:01:00.
  Batch   400  of    563.    Elapsed: 0:01:20.
  Batch   500  of    563.    Elapsed: 0:01:40.

  Average training loss: 0.48
  Training epcoh took: 0:01:52

Running Validation...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:161: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Accuracy: 0.91
  valid Loss: 0.46
  valid took: 0:00:04

======== Epoch 2 / 4 ========
Training...
  Batch   100  of    563.    Elapsed: 0:00:20.
  Batch   200  of    563.    Elapsed: 0:00:40.
  Batch   300  of    563.    Elapsed: 0:00:59.
  Batch   400  of    563.    Elapsed: 0:01:19.
  Batch   500  of    563.    Elapsed: 0:01:39.

  Average training loss: 0.19
  Training epcoh took: 0:01:51

Running Validation...
  Accuracy: 0.96
  valid Loss: 0.19
  valid took: 0:00:04

======== Epoch 3 / 4 ========
Training...
  Batch   100  of    563.    Elapsed: 0:00:20.
  Batch   200  of    563.    Elapsed: 0:00:39.
  Batch   300  of    563.    Elapsed: 0:00:59.
  Batch   400  of    563.    Elapsed: 0:01:19.
  Batch   500  of    563.    Elapsed: 0:01:39.

  Average training loss: 0.11
  Training epcoh took: 0:01:51

Running Validation...
  Accuracy: 0.97
  valid Loss: 0.16
  valid took: 0:00:04

======== Epoch 4 / 4 ========
Training...
  Batch   100  of    563.    Elapsed: 0:00:20.
  Batch   

In [92]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,valid Time
epoch,,,,,
1,0.48,0.46,0.91,0:01:52,0:00:04
2,0.19,0.19,0.96,0:01:51,0:00:04
3,0.11,0.16,0.97,0:01:51,0:00:04
4,0.04,0.12,0.98,0:01:51,0:00:04


In [93]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir+'bert-new-classification-model')
tokenizer.save_pretrained(output_dir)


# # Load a trained model and vocabulary that you have fine-tuned
# model = model_class.from_pretrained(output_dir)
# tokenizer = tokenizer_class.from_pretrained(output_dir)

# # Copy the model to the GPU.
# model.to(device)

Saving model to ./model_save/


('./model_save/vocab.txt',
 './model_save/special_tokens_map.json',
 './model_save/added_tokens.json')

In [97]:
with open('chinese_stopwords.txt', 'r') as f:
    #list of all stopwords
    stopwords = f.readlines()

with open('chinese_numbers_punctuation.txt', 'r') as f:
    #string with all chinese number and punctuations
    num_punct = f.read()
    
    
def get_cleaned_text(input_str):
    
    # to remove punctuations and numbers
    cleaned_words_list = input_str.translate(str.maketrans('','',string.punctuation + string.digits + num_punct + '\ufeff')).split()

    # to remove stop words
    tokens = []
    for word in cleaned_words_list:
        if word not in stopwords:
            tokens.append(word)
    return ' '.join(tokens)

def predict_news_class(sentence):

    import torch
    device = torch.device("cuda")

    from transformers import BertTokenizer, BertForSequenceClassification

    output_dir = './model_save/'
    input_ids = []
    attention_masks = []

    tokenizer_class = BertTokenizer
    model_class = BertForSequenceClassification

    tokenizer = tokenizer_class.from_pretrained(output_dir)

    process_sentence = get_cleaned_text(sentence)
    
    encoded_dict = tokenizer.encode_plus(
                        process_sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = median_sent_length,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',  # Return pytorch tensors.
                        truncation = True,
                          
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)
    # # Load a trained model and vocabulary that you have fine-tuned
    model = model_class.from_pretrained(output_dir+'bert-new-classification-model')

    # Copy the model to the GPU.
    model.to(device)

    model.eval()

    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(input_ids, token_type_ids=None, 
                        attention_mask=attention_masks)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    
    label_id = np.argmax(logits)
    labels_dict = {0: 'art', 1: 'computer', 2: 'economic', 3: 'education',  4: 'environment', 5: 'medical',6: 'military', 7: 'politics',8:'sports',9: 'traffic'}

    return labels_dict[label_id]

In [98]:
news1 = '班加羅爾是世界上交通最擁擠的城市，荷蘭位置技術專家TomTom發布了最新的交通指數。'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'traffic'

In [99]:


news1 = '香港抗議中國對待維吾爾人的抗議活動以暴力衝突告終'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'politics'

In [100]:
news1 = '金王朝，韋德-吉爾斯羅馬帝國化的中國王朝，由兩個不同的階段組成：西晉（西晉），從公元265年至316/317年統治中國； and（東晉），從公元317年至420年統治中國東晉被認為是六朝之一。'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'politics'

In [101]:
new1 = '速度慢，行程時間長和車輛排隊增加等特點'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'politics'

In [102]:
news1 = '薩欽是著名的板球運動員'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'sports'

In [103]:
news1 = '印度的識字率是5％，許多人貧窮'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'education'

In [104]:
news1 = '節約用水，挽救生命'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'medical'

In [105]:
news1 = '英超联赛已经同意在中国达成一项新的转播协议，以覆盖2020-21赛季的剩余时间。腾讯体育此前与PPTV的5.64亿英镑交易已终止，并已于本月初生效，此后已与腾讯体育签署了一项合同。中国观众将从本周末开始观看比赛。与PPTV的先前协议于2019年签署，有效期至2022年，在未收到3月份到期的1.6亿英镑付款后终止。新交易的财务细节尚未披露'
cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'sports'

In [106]:
news1 = '電暈導致全世界許多人喪生。疫苗仍處於準備階段'

cls = predict_news_class(news1)

cls

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'medical'